In [ ]:
import pandas as pd
import numpy as np
import logging as log

from data_prepare import getLogsFromDBByUserID

In [ ]:
df = pd.read_pickle('../data/173211-9739.pkl')
df['event'] = df['method'] + ':' + df['urlEntry']

In [ ]:
eventMap = {event: id for id, event in enumerate(df['event'].value_counts().keys())}
for id in eventMap.values(): 
    if id == 0:
        print(id)

In [ ]:
for id, event in enumerate(df['event'].value_counts().keys()):
    print((id, event))

In [ ]:
from datetime import datetime
# 增加日期
df['date'] = df['time'].apply(lambda x : datetime.fromtimestamp(x))
df.sort_values(by='time', ascending=True, inplace=True)
df = df.reset_index(drop=True)

In [ ]:
dates = df['date'].copy()
dates.sort_values(ascending=True, inplace=True)
# print(dates)
values = []
for i in range(1, len(dates)):
    values.append((dates[i] - dates[i-1]).total_seconds())
    if (dates[i] < dates[i-1]): 
        print(i, dates[i-1], dates[i])
v = np.asarray(values)
# values

In [ ]:
dates

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# threshold = 60*60*2
threshlodd = 30
data = []
for i in values:
#     if i < threshold and i >= threshlodd: 
    data.append(i)
# max(data)
# d = np.asarray(data)
# plt.hist(bins=300, x=d)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
months = df['date'].apply(lambda x : "%d-%d"%(x.year, x.month))
data = months.value_counts()
data.plot()

In [ ]:
df

In [ ]:
output = '../data/rewriteLog.log'
with open(output, 'w') as f:
    for logEntry in df.itertuples():
        f.write("%s %s %s %s %s %s %s\n" % (
            logEntry[8],
            logEntry[2],
            logEntry[3],
            logEntry[4],
            logEntry[5],
            logEntry[6],
            logEntry[7],
        ))

In [ ]:
from spellpy import spell

tau = 0.5
# input_dir = '../data/'
output_dir = '../data/output/'
log_format = '<Date> <Time> <Url> <Method> <Content> <Headers> <Name> <ADDR>'

parser = spell.LogParser(
    indir='./',
    outdir='./',
    log_format=log_format,
#     logmain=log_main,
    tau=tau,
)

parser.parse(output)


In [ ]:
df

In [ ]:
def deeplog_df_transfer(df, event_id_map):
  
    df = df[['date', 'event']]
    df['EventId'] = df['event'].apply(
        lambda e: event_id_map[e] if event_id_map.get(e) else -1)
    deeplog_df = df.set_index('date').resample(
        '30min').apply(_custom_resampler).reset_index()
    return deeplog_df


def _custom_resampler(array_like):
    return list(array_like)

def deeplog_file_generator(filename, df):
    with open(filename, 'w') as f:
        for event_id_list in df['EventId']:
            if (len(event_id_list) == 0):
                continue
            for event_id in event_id_list:
                f.write(str(event_id) + ' ')
            f.write('\n')

In [ ]:
df['event'] = df['method'] + ':' + df['urlEntry']
event_id_map = dict()
for i, event_id in enumerate(df['event'].unique(), 1):
    event_id_map[event_id] = i
event_id_map

## train

In [ ]:
epochs = range (1, 5)
losss = [0.015, 0.025, 0.045, 0.065]
accs = [0.085, 0.075, 0.045, 0.015]

import matplotlib.pyplot as plt
fig = plt.figure()

ax1 = fig.add_subplot(2, 1, 1)
ax2 = fig.add_subplot(2, 1, 2)
ax1.plot(epochs, losss, 'k')
ax2.plot(accs, losss)


In [1]:
urls = []
with open("../data/urls_origin.txt", 'r') as f:
    for line in f:
        urls.append(line.strip())
urls.__len__()

275974

In [2]:
from modules.url_match import UrlMatcher, UrlEntry
from sql.db import DB as db

entrys = db.queryUrlEntrys()

# entrys
# matcher = UrlMatcher()

In [3]:
urlEntrys = [UrlEntry(paraSet=(entry[2].split('/')[1], entry[1], entry[2])) for entry in entrys]
matcher = UrlMatcher(urlEntrys)

In [5]:
dic = set()
with open('../data/all_urls.csv', 'r') as f:
    first = True
    for line in f:
        if first:
            first = False
            continue
        ss = line.strip().split(',')
        module = ss[-1]
        url = line.strip().strip(','+module)
        url = '/' + url.strip('/')
        try:
            entry = matcher.convert(url, module)
            dic.add(entry)    
        except Exception as e:
            print(e)
            

In [11]:
# '/hr/thrpeople' in dic
matcher.convert('/hr/tpayusers', 'POST')